<a href="https://colab.research.google.com/github/Tejaswidarsi/Feature-selection/blob/main/FeatureSelection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
# Import the necessary libraries
import pandas as pd
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression
# Import RandomForestRegressor and necessary metrics
from sklearn.ensemble import RandomForestRegressor  # Import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score # Import metrics for regression
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer # Import SimpleImputer for handling missing values


##Missing Values Ratio


####1. Diabetes Dataset:

Loading the dataset  and checking the missing values before filtering

In [57]:
diabetes_dataset = pd.read_csv("/content/Diabetes Missing Data (1).csv")
diabetes_dataset.isnull().sum()

,0
Pregnant,0
Glucose,5
Diastolic_BP,35
Skin_Fold,227
Serum_Insulin,374
BMI,11
Diabetes_Pedigree,0
Age,0
Class,0


Filtering columns based on missing values

In [58]:
missing_ratio = diabetes_dataset.isnull().mean()
diabetes_filtered = diabetes_dataset.loc[:, missing_ratio <= 0.30]
diabetes_filtered.isnull().sum()

,0
Pregnant,0
Glucose,5
Diastolic_BP,35
Skin_Fold,227
BMI,11
Diabetes_Pedigree,0
Age,0
Class,0


####2.Melbourne Housing Dataset:


In [59]:
mel_dataset = pd.read_csv("/content/melbourne_housing_raw.csv")
mel_dataset.info()
mel_dataset.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34857 entries, 0 to 34856
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         34857 non-null  object 
 1   Rooms          34857 non-null  int64  
 2   Type           34857 non-null  object 
 3   Price          27247 non-null  float64
 4   Method         34857 non-null  object 
 5   SellerG        34857 non-null  object 
 6   Date           34857 non-null  object 
 7   Distance       34856 non-null  float64
 8   Postcode       34856 non-null  float64
 9   Bedroom2       26640 non-null  float64
 10  Bathroom       26631 non-null  float64
 11  Car            26129 non-null  float64
 12  Landsize       23047 non-null  float64
 13  BuildingArea   13742 non-null  float64
 14  YearBuilt      15551 non-null  float64
 15  CouncilArea    34854 non-null  object 
 16  Lattitude      26881 non-null  float64
 17  Longtitude     26881 non-null  float64
 18  Region

,0
Suburb,0
Rooms,0
Type,0
Price,7610
Method,0
SellerG,0
Date,0
Distance,1
Postcode,1
Bedroom2,8217


In [60]:
missing_ratio = mel_dataset.isnull().mean()
melbourne_filtered = mel_dataset.loc[:, missing_ratio <= 0.20]
melbourne_filtered.isnull().sum()


,0
Suburb,0
Rooms,0
Type,0
Method,0
SellerG,0
Date,0
Distance,1
Postcode,1
CouncilArea,3
Regionname,3


##High Correlation Filter

####3.Diabetes Dataset:

Find pairs of features with correlation > 0.8, remove one from each pair, then assess model accuracy.

In [61]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer

# Assuming 'diabetes_dataset' is already loaded, and 'Class' is the target column
# Step 1: Split dataset into features and target variable
X = diabetes_dataset.drop('Class', axis=1)
y = diabetes_dataset['Class']

# Step 2: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Step 3: Impute missing values (mean strategy)
imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# Step 4: Train Logistic Regression and assess accuracy before removing correlated features
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train, y_train)

y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Accuracy before removing correlated features
train_acc_before = accuracy_score(y_train, y_pred_train)
test_acc_before = accuracy_score(y_test, y_pred_test)

# Print results before feature removal
print(f"Accuracy before feature removal - Train: {train_acc_before}, Test: {test_acc_before}")

# Step 5: Removing highly correlated features
# Calculate correlation matrix and identify highly correlated features
X_train_df = pd.DataFrame(X_train, columns=X.columns) # Convert X_train back to DataFrame for correlation calculation
corr_matrix = X_train_df.corr().abs()
high_corr_features = [column for column in corr_matrix.columns if any(corr_matrix[column] > 0.8) and column != corr_matrix.index[corr_matrix[column].argmax()]] # Exclude the feature itself from the list


X_train_filtered = X_train_df.drop(columns=high_corr_features)
X_test_filtered = pd.DataFrame(X_test, columns=X.columns).drop(columns=high_corr_features)

# Impute again for filtered data (if necessary, depending on filtering process)
X_train_filtered = imputer.fit_transform(X_train_filtered)
X_test_filtered = imputer.transform(X_test_filtered)

# Step 6: Train the model and assess accuracy after removing correlated features
model.fit(X_train_filtered, y_train)
y_pred_train_filtered = model.predict(X_train_filtered)
y_pred_test_filtered = model.predict(X_test_filtered)

# Accuracy after removing correlated features
train_acc_after = accuracy_score(y_train, y_pred_train_filtered)
test_acc_after = accuracy_score(y_test, y_pred_test_filtered)

# Print results after feature removal
print(f"Highly correlated features removed: {high_corr_features}")
print(f"Accuracy after feature removal - Train: {train_acc_after}, Test: {test_acc_after}")

Accuracy before feature removal - Train: 0.7802607076350093, Test: 0.7402597402597403
Highly correlated features removed: []
Accuracy after feature removal - Train: 0.7802607076350093, Test: 0.7402597402597403


####4. Melbourne Housing Dataset:

Similarly, remove features with a correlation greater than 0.85 and evaluate the model's performance for price prediction.

In [62]:
mf = pd.DataFrame(melbourne_filtered, columns=melbourne_filtered.columns)
num_features = melbourne_filtered.select_dtypes(include=['number']).columns
corr_matrix = melbourne_filtered[num_features].corr().abs()
high_corr_features = [column for column in corr_matrix.columns if any(corr_matrix[column] > 0.85)]
mel_filtered = melbourne_filtered.drop(columns=high_corr_features)
print(mel_filtered.head())


       Suburb Type Method SellerG     Date         CouncilArea  \
0  Abbotsford    h     SS  Jellis   3/9/16  Yarra City Council   
1  Abbotsford    h      S  Biggin  3/12/16  Yarra City Council   
2  Abbotsford    h      S  Biggin   4/2/16  Yarra City Council   
3  Abbotsford    u     VB  Rounds   4/2/16  Yarra City Council   
4  Abbotsford    h     SP  Biggin   4/3/17  Yarra City Council   

              Regionname  
0  Northern Metropolitan  
1  Northern Metropolitan  
2  Northern Metropolitan  
3  Northern Metropolitan  
4  Northern Metropolitan  


##Low Variance Filter

####5.Diabetes Dataset:

Remove features with very low variance and retrain the model

In [63]:
from sklearn.feature_selection import VarianceThreshold
selector = VarianceThreshold(threshold=0.01)
diabetes_filtered = selector.fit_transform(diabetes_dataset)

####6.Melbourne Housing Dataset:

Apply the low variance filter and evaluate its impact on housing price prediction

In [64]:
selector = VarianceThreshold(threshold=0.01)
melbourne_filtered = selector.fit_transform(mel_dataset[num_features])

##Forward Feature Selection

####7. Diabetes Dataset:

Use forward selection to select the best features for logistic regression, and determine the optimal number of features.

In [65]:
imputer = SimpleImputer(strategy='mean') # Create an imputer instance with strategy='mean'
X_diabetes_imputed = imputer.fit_transform(X) # Fit and transform the imputer on X_diabetes
X_diabetes_imputed = pd.DataFrame(X_diabetes_imputed, columns=X.columns) # Convert back to DataFrame

X_train, X_test, y_train, y_test = train_test_split(X_diabetes_imputed, y, test_size=0.3, random_state=42)
# Use the imputed data for train_test_split


log_reg = LogisticRegression(max_iter=1000)
sfs_diabetes = SequentialFeatureSelector(log_reg, n_features_to_select="auto", direction='forward')
sfs_diabetes.fit(X_train, y_train)
diabetes_selected_features = sfs_diabetes.get_support()
print("Diabetes Selected Features: ", diabetes_selected_features)

Diabetes Selected Features:  [False  True False False False  True  True  True]


In [66]:
melbourne_data = mel_dataset.dropna(subset=['Price']).sample(frac=0.2, random_state=42)

# Define feature matrix X and target vector y
X = melbourne_data.drop('Price', axis=1)
y = melbourne_data['Price']

# Optional: Drop columns with high missing values or that are unlikely to be useful
X = X.drop(columns=['BuildingArea', 'YearBuilt', 'SellerG', 'Date', 'Lattitude', 'Longtitude'], errors='ignore')

categorical_features = X.select_dtypes(include=['object']).columns

# ----> One-hot encode categorical features <----
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_data = encoder.fit_transform(X[categorical_features])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_features))

# ----> Concatenate encoded features with numerical features <----
X = X.drop(categorical_features, axis=1)
X = pd.concat([X.reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)

# ----> Impute missing values <----
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

# Combine X and y for sampling
combined_data = pd.DataFrame(X)  # Convert to DataFrame for easier handling
combined_data['Price'] = y.reset_index(drop=True)

# Split the combined data into training and testing sets
X_train, X_test = train_test_split(combined_data, test_size=0.3, random_state=42)

# Sample from the training data
X_sample = X_train.sample(frac=0.2, random_state=42)
y_sample = X_sample['Price']
X_sample = X_sample.drop(columns=['Price'])  # Drop the target column

# Step 1: Initialize the Linear Regression model
lin_reg = LinearRegression()

# Step 2: Implement Forward Feature Selection with reduced parameters
sfs = SequentialFeatureSelector(lin_reg, n_features_to_select=5, direction='forward', scoring='neg_mean_squared_error', cv=2)

# Fit the selector to the sample of the training data for quicker evaluation
sfs.fit(X_sample, y_sample)

# Step 3: Get the selected features
selected_features = sfs.get_support()

# Step 4: Prepare X_train and y_train for model training
X_train_selected = X_train.drop(columns=['Price']).iloc[:, selected_features]
y_train = X_train['Price']  # Get y_train from the original X_train

# Ensure the same selection is applied to X_test
X_test_selected = X_test.drop(columns=['Price']).iloc[:, selected_features]

# Fit the Linear Regression model using the selected features
lin_reg.fit(X_train_selected, y_train)

# Step 5: Make predictions and evaluate the model
y_pred_train = lin_reg.predict(X_train_selected)
y_pred_test = lin_reg.predict(X_test_selected)

train_mse = mean_squared_error(y_train, y_pred_train)
test_mse = mean_squared_error(X_test['Price'], y_pred_test)

# Print results
print(f"Selected Features: {original_columns[selected_features]}")

Selected Features: Index([0, 1, 319, 364, 365], dtype='object')


##Backward Feature Elimination

####9. Diabetes Dataset:
Perform backward feature elimination on the diabetes dataset using
a decision tree classifier, removing the least important features one by one,

In [67]:
import pandas as pd
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score

# Load the diabetes dataset
diabetes_data = load_diabetes()
X = pd.DataFrame(diabetes_data.data, columns=diabetes_data.feature_names)
y = (diabetes_data.target > diabetes_data.target.mean()).astype(int)  # Convert to binary outcome

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a decision tree classifier
decision_tree = DecisionTreeClassifier(random_state=42)

# Implement backward feature elimination
selector = RFE(decision_tree, n_features_to_select=1)  # We want to eliminate features one by one
selector.fit(X_train, y_train)

# Get the ranking of features
feature_ranking = selector.ranking_
features = X.columns.tolist()

# Create a DataFrame for feature ranking
ranking_df = pd.DataFrame({'Feature': features, 'Ranking': feature_ranking})
print("Feature Rankings:\n", ranking_df.sort_values('Ranking'))

# Select features
selected_features = X.columns[selector.support_].tolist()
print("Selected Features after Backward Elimination:", selected_features)

# Train the model with the selected features
decision_tree.fit(X_train[selected_features], y_train)

# Predict on the test set
y_pred = decision_tree.predict(X_test[selected_features])

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy of the model with selected features: {accuracy:.2f}')


Feature Rankings:
   Feature  Ranking
2     bmi        1
8      s5        2
4      s1        3
9      s6        4
3      bp        5
5      s2        6
6      s3        7
0     age        8
7      s4        9
1     sex       10
Selected Features after Backward Elimination: ['bmi']
Accuracy of the model with selected features: 0.67


####10. Melbourne Housing Dataset: Apply backward feature elimination on the Melbourne
housing dataset using a random forest model, and analyze how removing the least
important features one at a time impacts the accuracy of price predictions.

In [68]:
random_forest = RandomForestRegressor(random_state=42)

# Implement backward feature elimination
selector = RFE(random_forest, n_features_to_select=1)  # We want to eliminate features one by one
selector.fit(X_train, y_train)

# Get the ranking of features
feature_ranking = selector.ranking_
features = X.columns.tolist()

ranking_df = pd.DataFrame({'Feature': features, 'Ranking': feature_ranking})
print("Feature Rankings:\n", ranking_df.sort_values('Ranking'))

# Select features
selected_features = X.columns[selector.support_].tolist()
print("Selected Features after Backward Elimination:", selected_features)

# Train the model with the selected features
random_forest.fit(X_train[selected_features], y_train)

# Predict on the test set
y_pred = random_forest.predict(X_test[selected_features])

# Calculate metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error with selected features: {mse:.2f}')
print(f'R² Score with selected features: {r2:.2f}')

Feature Rankings:
   Feature  Ranking
8      s5        1
2     bmi        2
3      bp        3
5      s2        4
0     age        5
9      s6        6
4      s1        7
6      s3        8
7      s4        9
1     sex       10
Selected Features after Backward Elimination: ['s5']
Mean Squared Error with selected features: 0.21
R² Score with selected features: 0.13


##Random Forest

In [69]:
import pandas as pd
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load the diabetes dataset

random_forest = RandomForestClassifier(random_state=42)

# Train the model
random_forest.fit(X_train, y_train)

# Get feature importances
importances = random_forest.feature_importances_

# Create a DataFrame for feature importance
importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Select the top 5 most important features
top_features = importance_df.head(5)['Feature'].tolist()
print("Top 5 Important Features:", top_features)

# Train the model with the selected features
random_forest.fit(X_train[top_features], y_train)

# Predict on the test set
y_pred = random_forest.predict(X_test[top_features])

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy of the model with top 5 features: {accuracy:.2f}')


Top 5 Important Features: ['bmi', 's5', 'bp', 's6', 's2']
Accuracy of the model with top 5 features: 0.73


In [70]:
random_forest = RandomForestRegressor(random_state=42)

# Train the model
random_forest.fit(X_train, y_train)

# Get feature importances
importances = random_forest.feature_importances_

# Create a DataFrame for feature importance
importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Optionally, you can remove the least important features, e.g., keep top k features
k = 10  # You can adjust this based on your analysis
top_features = importance_df.head(k)['Feature'].tolist()
print("Top Features for Housing Prices:", top_features)
random_forest.fit(X_train[top_features], y_train)

# Predict on the test set
y_pred = random_forest.predict(X_test[top_features])

# Calculate metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error with selected features: {mse:.2f}')
print(f'R² Score with selected features: {r2:.2f}')


Top Features for Housing Prices: ['s5', 'bmi', 'bp', 's2', 'age', 's6', 's1', 's3', 's4', 'sex']
Mean Squared Error with selected features: 0.19
R² Score with selected features: 0.24
